In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# -----------------------------
# 1. داده مصنوعی
# -----------------------------
np.random.seed(42)
X = 2 * np.random.rand(50)
Y = 4 + 3 * X + np.random.randn(50)

# -----------------------------
# 2. گرادیان نزولی
# -----------------------------
a, b = 0.0, 0.0
lr = 0.1
epochs = 30

# تاریخچه پارامترها و هزینه
a_hist, b_hist = [a], [b]
cost_hist = [np.sum((Y - (a*X + b))**2)/(2*len(X))]  # مقدار اولیه

for _ in range(epochs):
    y_pred = a * X + b
    da = -2 * np.sum(X * (Y - y_pred)) / len(X)
    db = -2 * np.sum(Y - y_pred) / len(X)
    a -= lr * da
    b -= lr * db
    a_hist.append(a)
    b_hist.append(b)
    cost_hist.append(np.sum((Y - (a*X + b))**2)/(2*len(X)))

# -----------------------------
# 3. سطح تابع هزینه
# -----------------------------
a_vals = np.linspace(min(a_hist)*0.8, max(a_hist)*1.2, 50)
b_vals = np.linspace(min(b_hist)*0.8, max(b_hist)*1.2, 50)
A, B = np.meshgrid(a_vals, b_vals)
Z = np.zeros_like(A)
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        Z[i,j] = np.sum((Y - (A[i,j]*X + B[i,j]))**2)/(2*len(X))

# -----------------------------
# 4. آماده‌سازی نمودارها
# -----------------------------
fig = plt.figure(figsize=(18,6))

# (الف) خط روی داده‌ها
ax1 = fig.add_subplot(1,3,1)
scatter = ax1.scatter(X, Y, color='blue', label='Data')
line, = ax1.plot([], [], 'r-', label='Prediction')
ax1.set_xlim(0, 2.5)
ax1.set_ylim(0, 12)
ax1.set_title("Linear Fit")
ax1.legend()

# (ب) نمودار هزینه
ax2 = fig.add_subplot(1,3,2)
ax2.set_xlim(0, epochs)
ax2.set_ylim(0, max(cost_hist)*1.1)
ax2.set_title("Cost Function")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Cost (MSE)")
cost_line, = ax2.plot([], [], 'purple', linewidth=2)

# (ج) 3D سطح هزینه
ax3 = fig.add_subplot(1,3,3, projection='3d')
surf = ax3.plot_surface(A, B, Z, cmap='viridis', alpha=0.6)
param_point, = ax3.plot([], [], [], 'ro', markersize=6)
path_line, = ax3.plot([], [], [], 'r--', alpha=0.3)
ax3.set_xlabel('a (slope)')
ax3.set_ylabel('b (intercept)')
ax3.set_zlabel('Cost')
ax3.set_title('3D Cost Surface')

# -----------------------------
# 5. init و update
# -----------------------------
def init():
    line.set_data([], [])
    cost_line.set_data([], [])
    scatter.set_color(['blue']*len(X))
    param_point.set_data([], [])
    param_point.set_3d_properties([])
    path_line.set_data([], [])
    path_line.set_3d_properties([])
    return line, cost_line, scatter, param_point, path_line

def update(i):
    # خط روی داده‌ها
    y_pred_points = a_hist[i]*X + b_hist[i]
    line.set_data(X, y_pred_points)
    errors = np.abs(Y - y_pred_points)
    scatter.set_color(plt.cm.Reds(errors / max(errors)))

    # نمودار هزینه
    cost_line.set_data(np.arange(0, i+1), cost_hist[:i+1])

    # نقطه و مسیر روی سطح هزینه
    cost_now = cost_hist[i]
    param_point.set_data([a_hist[i]], [b_hist[i]])
    param_point.set_3d_properties([cost_now])

    path_line.set_data(a_hist[:i+1], b_hist[:i+1])
    path_line.set_3d_properties(cost_hist[:i+1])

    # چرخش 3D
    ax3.view_init(elev=30, azim=i*12)  # زاویه چرخش تغییر می‌کند

    ax1.set_title(f"Epoch {i}, a={a_hist[i]:.2f}, b={b_hist[i]:.2f}")
    return line, cost_line, scatter, param_point, path_line

# -----------------------------
# 6. اجرای انیمیشن
# -----------------------------
anim = FuncAnimation(fig, update, frames=len(a_hist), init_func=init, blit=False, interval=400)
plt.close()
HTML(anim.to_jshtml())


Output hidden; open in https://colab.research.google.com to view.